### 붓꽃 품종 분류
- 목표 : 붓꽃의 3개 품종을 분류하기
- 데이터셋 : 내장 데이터셋
- 피쳐 : 4개
- 타겟 : 품종 1개
- 학습 : 지도학습 > 분류

[1] 데이터 준비

In [98]:
# 모듈 로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [99]:
# 내장 데이터셋 로딩
data = load_iris(as_frame = True)

In [100]:
# Bunch 인스턴스 => dict와 유사한 형태
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [101]:
featureDF = data['data']
targetSR = data['target']

In [102]:
featureDF.shape, targetSR.shape

((150, 4), (150,))

In [103]:
featureDF.head(1), targetSR.head(1)

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                5.1               3.5                1.4               0.2,
 0    0
 Name: target, dtype: int64)

[2] 학습을 위한 데이터셋 준비 => 학습용, 검증용, 테스트용

In [104]:
# 학습용 & 테스트용 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetSR, stratify=targetSR)

In [105]:
# 학습용 & 검증용 분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train)

In [106]:
print(f"Train DS : {X_train.shape[0]} {X_train.shape[0]/featureDF.shape[0]}%")
print(f"Val DS : {X_val.shape[0]} {X_val.shape[0]/featureDF.shape[0]:.2f}%")
print(f"Test DS : {X_test.shape[0]} {X_test.shape[0]/featureDF.shape[0]:.2f}%")

Train DS : 84 0.56%
Val DS : 28 0.19%
Test DS : 38 0.25%


[3] 교차검증 방식

In [107]:
# 모듈 로딩
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [108]:
# 모델 인스턴스 생성
dtc_model = DecisionTreeClassifier()


# KFold 인스턴스 생성 [기본 k=5]
kfold = KFold()

In [109]:
# [3-1] KFold 기반 ------------------
# 정확도 저장 리스트
accuracys = []

# K번 만큼 K개 데이터셋으로 학습 진행
# -> K등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(kfold.split(featureDF)):
    # print(f"train_index : {train_index.tolist()}")

    # X_train, X_val 데이터셋 설정
    X_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val, y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    # 학습 진행
    dtc_model.fit(X_train, y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train, y_train)
    val_acc = dtc_model.score(X_val, y_val)
    
    accuracys.append([train_acc, val_acc])
    print(f"[{idx + 1}번째] Train 정확도 : {train_acc}, Val 정확도 : {val_acc}")

[1번째] Train 정확도 : 1.0, Val 정확도 : 1.0
[2번째] Train 정확도 : 1.0, Val 정확도 : 1.0
[3번째] Train 정확도 : 1.0, Val 정확도 : 0.9
[4번째] Train 정확도 : 1.0, Val 정확도 : 0.9333333333333333
[5번째] Train 정확도 : 1.0, Val 정확도 : 0.8


In [110]:
# 평균 계산
train_mean = sum([i[0] for i in accuracys]) / kfold.n_splits
test_mean = sum([i[1] for i in accuracys]) / kfold.n_splits
print(f"Train 정확도 : {train_mean}     Val 정확도 : {test_mean:.2f}")

Train 정확도 : 1.0     Val 정확도 : 0.93


In [111]:
# [3-2] StratifiedKFold 기반 : 정답/레이블/타겟의 비율을 고려해서 데이터 나눔
# 정확도 저장 리스트
accuracys = []

skfold = StratifiedKFold()

# K번 만큼 K개 데이터셋으로 학습 진행
# -> K등분 후 학습용 데이터셋 인덱스, 검증용 데이터셋 인덱스
for idx, (train_index, val_index) in enumerate(skfold.split(featureDF, targetSR)):
    # print(f"train_index : {train_index.tolist()}")

    # X_train, X_val 데이터셋 설정
    X_train, y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val, y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    # 학습 진행
    dtc_model.fit(X_train, y_train)
    
    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train, y_train)
    val_acc = dtc_model.score(X_val, y_val)

    accuracys.append([train_acc, val_acc])
    print(f"[{idx + 1}번째] Train 정확도 : {train_acc}, Val 정확도 : {val_acc}")

[1번째] Train 정확도 : 1.0, Val 정확도 : 0.9666666666666667
[2번째] Train 정확도 : 1.0, Val 정확도 : 0.9666666666666667
[3번째] Train 정확도 : 1.0, Val 정확도 : 0.9
[4번째] Train 정확도 : 1.0, Val 정확도 : 0.9666666666666667
[5번째] Train 정확도 : 1.0, Val 정확도 : 1.0


In [112]:
# 평균 계산
train_mean = sum([i[0] for i in accuracys]) / skfold.n_splits
test_mean = sum([i[1] for i in accuracys]) / skfold.n_splits
print(f"Train 정확도 : {train_mean}     Val 정확도 : {test_mean:.2f}")

Train 정확도 : 1.0     Val 정확도 : 0.96


- 교차검증 및 성능평가 동시 진행 함수
    * => cross_val_score, cross_val_predict
    * => cross_validate

In [113]:
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate

In [114]:
# [1] 전체 DS ==> 학습용과 테스트용 DS 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR,
                                                    stratify=targetSR)

In [115]:
# cross_val_predict
predict = cross_val_predict(dtc_model, featureDF, targetSR, cv = 3)

In [116]:
print(f"predict : {predict}")

predict : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [117]:
# cross_val_score
cross_val_score(dtc_model, featureDF, targetSR)

array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])

In [118]:
# cross_validate
result = cross_validate(dtc_model, featureDF, targetSR,
                        return_train_score=True,
                        return_estimator=True)
result

{'fit_time': array([0.00126696, 0.0007689 , 0.0007031 , 0.00066996, 0.00065303]),
 'score_time': array([0.00092793, 0.00054002, 0.00048089, 0.00050688, 0.00046897]),
 'estimator': [DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier()],
 'test_score': array([0.96666667, 0.96666667, 0.9       , 0.93333333, 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [119]:
resultDF = pd.DataFrame(result).loc[:, ['test_score', 'train_score']]
resultDF

,test_score,train_score
0,0.966667,1.0
1,0.966667,1.0
2,0.900000,1.0
3,0.933333,1.0
4,1.000000,1.0


In [120]:
model = result['estimator'][2]

model.score(featureDF, targetSR)

0.98

In [121]:
model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
# 이게 전부 Desicion Tree 하이퍼 파라미터